This project is has some objective
* Build a custom model
*	Use different pretrained models(alexnet, vgg16,resnet18, resnet36)
*	Use different optimization algorithm(sgd,sgd with momentum,adam)
*	Use different learning rate.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
#cuda helps in utilizing the GPU for faster computation
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )
device

Now we will import the data

In [ ]:
Imagesize = (64,64)
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(Imagesize),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                ])

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='../input/flowers-recognition/flowers/flowers',transform = transform)
dataset

In [ ]:
print("dataset has the following classes ",dataset.classes)
print(type(dataset))
num_classes = len(dataset.classes)
print(num_classes)

In [ ]:
#now splitting data in trainset and testset
trainset, testset = torch.utils.data.random_split(dataset, [3900,423])


* num_workers, which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that CPU computations are efficiently managed, i.e. that the bottleneck is indeed the neural network's forward and backward operations on the GPU (and not data generation).

In [ ]:
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=16,num_workers=3, shuffle=True) 
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=16,num_workers=3, shuffle=True)

# Building Neural Network Architecture**

* **Convnet Architecture - (((((Conv+Relu)*n)+pool)*m) + F.C.)**
* **Image size after convolution - ((W-F+2P)/S) + 1**
* **Conv2d outputs a tensor of shape [batch_size, n_features_conv, height, width]**

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=5):
        super(ConvNet,self).__init__()
        self.layer1 = nn.Sequential(
                    nn.Conv2d(3,16, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(16),
                    nn.ReLU(),
        )           #output - 16*64*64 image
        
        self.layer2 = nn.Sequential(
                    nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                    #output - 32*64*64
                    nn.MaxPool2d(kernel_size=2, stride=2)
                    #output - 32*32*32
        )           
        
        self.layer3 = nn.Sequential(
                    nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    #output - 64*32*32
        )
        self.layer4 = nn.Sequential(
                    nn.Conv2d(64,128, kernel_size=5, stride=1, padding=2),
                    nn.BatchNorm2d(128),
                    nn.ReLU(),
                    #output - 128*32*32
                    nn.MaxPool2d(kernel_size=2, stride=2)
                    #output - 128*16*16
        )
        self.fc1 = nn.Linear(128*16*16, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return F.log_softmax(out,dim=1)

In [ ]:
model = ConvNet(num_classes).to(device)
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
num_epochs = 10

In [ ]:
def train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, model, trainloader, testloader, criterion, optimizer):
    total_step = len(trainloader)
    for epoch in range(num_epochs):
        trainAcc = 0
        testAcc = 0
        for i, data in enumerate(trainloader,0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            trainLoss = criterion(outputs, labels)
            optimizer.zero_grad()                                        
            trainLoss.backward()                                        
            optimizer.step()  
            preds = outputs.data.max(dim=1,keepdim=True)[1]
            trainAcc += preds.eq(labels.data.view_as(preds)).cpu().sum()
        trainAcc = trainAcc/len(trainloader.dataset) * 100
        
        # For testset
        for i, data in enumerate(testloader):
            model.eval()
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            testLoss = criterion(outputs, labels)
            preds = outputs.data.max(dim=1,keepdim=True)[1]
            testAcc += preds.eq(labels.data.view_as(preds)).cpu().sum()
        testAcc = testAcc/len(testloader.dataset) * 100
        print("Epoch {} =>  loss : {trainLoss:.2f};   Accuracy : {trainAcc:.2f}%;   test_loss : {testLoss:.2f};   test_Accuracy : {testAcc:.2f}%".format(epoch+1, trainLoss=trainLoss.item(), trainAcc=trainAcc, testLoss=testLoss.item(), testAcc=testAcc))
        TrainLoss.append(trainLoss)
        TrainAcc.append(trainAcc)
        TestLoss.append(testLoss)
        TestAcc.append(testAcc)

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, model, trainloader, testloader, criterion, optimizer)

With other optimizers --SGD

In [ ]:
model_sgd = ConvNet(num_classes).to(device)
model_sgd

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_sgd.parameters(), lr=0.01)

In [ ]:
num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, model_sgd, trainloader, testloader, criterion, optimizer)

With other Optimizer (SGD With Momentum)

In [ ]:
model_sgd_momentum = ConvNet(num_classes).to(device)
model_sgd_momentum

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_sgd_momentum.parameters(), lr=0.01, momentum=0.9)

In [ ]:
num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, model_sgd_momentum, trainloader, testloader, criterion, optimizer)

using AlexNet

In [ ]:
alexnet = torchvision.models.alexnet(pretrained=True)
alexnet

In [ ]:
alexnet.classifier[6].out_features = 5
for param in alexnet.features.parameters(): 
    param.requires_grad = False

alexnet = alexnet.cuda()

In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, alexnet, trainloader, testloader, criterion, optimizer)

using VGG16

In [ ]:
vgg = torchvision.models.vgg16(pretrained=True)
vgg

In [ ]:
vgg.classifier[6].out_features = 5
for param in vgg.features.parameters(): 
    param.requires_grad = False

vgg = vgg.cuda()

In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg.parameters(), lr=0.001)

In [ ]:

num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, vgg, trainloader, testloader, criterion, optimizer)

Using Resnet18

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained= True)
resnet18

In [ ]:
fr = resnet18.fc.in_features
resnet18.fc = nn.Linear(fr, 5)

resnet18 = resnet18.cuda()

In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.001)

In [ ]:

num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, resnet18, trainloader, testloader, criterion, optimizer)

Resnet34

In [ ]:
resnet34 = torchvision.models.resnet34(pretrained=True)
resnet34

In [ ]:
resnet34

In [ ]:
fr = resnet34.fc.in_features
resnet34.fc = nn.Linear(fr, 5)
resnet34 = resnet34.cuda()

In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet34.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

In [ ]:
train(TrainLoss,TrainAcc, TestLoss, TestAcc, num_epochs, resnet34, trainloader, testloader, criterion, optimizer)